In [ ]:
import mxnet as mx
from mxnet import nd, autograd, gluon
from mxnet.gluon import nn

In [ ]:
a = mx.sym.Variable('a')
b = mx.sym.Variable('b')
c = mx.sym.Variable('c')

# 定义计算关系
d = (a + b) * c

# 基本操作
## 指定每个输入符号对应的输出

In [ ]:
input_args = {
    'a' : nd.array([1]),
    'b' : nd.array([2]),
    'c' : nd.array([3])
}

`a`, `b`, `c`, d` 定义的只是计算关系，执行计算（包括申请内存等操作）需要 Executor 用 `bind()` 函数指定输入，`d` 为输出。

In [ ]:
executor = d.bind(ctx=mx.cpu(), args=input_args)

## 执行计算

In [ ]:
executor.forward()

In [ ]:
executor.outputs[0].asnumpy()

符号式计算是函数式编程的思路，计算是延迟的（Lazy），符号变量只是用来把数据的 NDArray 和 Symbol 绑定起来，实际的计算发生在 Executor 调用时。

### 初始化一个变量用来保存关于 $a$ 的梯度

In [ ]:
grad_a = mx.nd.empty(1)

### 在 `bind()` 函数中指定要求梯度的变量

In [ ]:
executor = d.bind(
    ctx=mx.cpu(0),
    args = input_args,
    args_grad = {'a' : grad_a}
)

因为梯度是传播的，所以最后的输出节点的梯度需要指定，这里用$1$：

In [ ]:
executor.backward(out_grads=mx.nd.ones(1))

In [ ]:
# 计算出梯度，将自动刷新在 grad_a 中
grad_a

`mx.kvstore` 模块支持分布式计算。

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt


def cos_curve(x):
    return 0.25*np.sin(2*x*np.pi+0.5*np.pi) + 0.5


np.random.seed(123)
samples = []
labels = []

sample_density = 50
for i in range(sample_density):
    x1, x2 = np.random.random(2)
    bound = cos_curve(x1)
    if bound - 0.1 < x2 <= bound + 0.1:
        continue
    else:
        samples.append((x1, x2))
        if x2 > bound:
            labels.append(1)
        else:
            labels.append(0)

with open('data.pkl', 'wb') as f:
    pickle.dump((samples, labels), f)

for i, sample in enumerate(samples):
    plt.plot(sample[0], sample[1],
             'o' if labels[i] else '^',
             mec='r' if labels[i] else 'b',
             mfc='none',
             markersize=10)

x1 = np.linspace(0, 1)
plt.plot(x1, cos_curve(x1), 'k--')
plt.show()

In [ ]:
import pickle
import logging
import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the network
data = mx.sym.Variable('data')
fc1 = mx.sym.FullyConnected(data=data, name='fc1', num_hidden=2)
sigmoid1 = mx.sym.Activation(data=fc1, name='sigmoid1', act_type='sigmoid')
fc2 = mx.sym.FullyConnected(data=sigmoid1, name='fc2', num_hidden=2)
mlp = mx.sym.SoftmaxOutput(data=fc2, name='softmax')

shape = {'data': (2, )}
mlp_dot = mx.viz.plot_network(symbol=mlp, shape=shape)
mlp_dot.render('simple_mlp.gv', view=True)

# Load data & train the model
with open('data.pkl', 'rb') as f:
    samples, labels = pickle.load(f)

logging.getLogger().setLevel(logging.DEBUG)

batch_size = len(labels)
samples = np.array(samples)
labels = np.array(labels)

train_iter = mx.io.NDArrayIter(samples, labels, batch_size)

model = mx.model.FeedForward.create(
    symbol=mlp, X=train_iter, num_epoch=1000, learning_rate=0.1, momentum=0.99)
'''
# Alternative interface to train the model
model = mx.model.FeedForward(
    symbol=mlp,
    num_epoch=1000,
    learning_rate=0.1,
    momentum=0.99)
model.fit(X=train_iter)
'''

print(model.predict(mx.nd.array([[0.5, 0.5]])))

# Visualize result
X = np.arange(0, 1.05, 0.05)
Y = np.arange(0, 1.05, 0.05)
X, Y = np.meshgrid(X, Y)

grids = mx.nd.array(
    [[X[i][j], Y[i][j]] for i in range(X.shape[0]) for j in range(X.shape[1])])
grid_probs = model.predict(grids)[:, 1].reshape(X.shape)

fig = plt.figure('Sample Surface')
ax = fig.gca(projection='3d')

ax.plot_surface(
    X, Y, grid_probs, alpha=0.15, color='k', rstride=2, cstride=2, lw=0.5)
samples0 = samples[labels == 0]
samples0_probs = model.predict(samples0)[:, 1]
samples1 = samples[labels == 1]
samples1_probs = model.predict(samples1)[:, 1]

ax.scatter(
    samples0[:, 0], samples0[:, 1], samples0_probs, c='b', marker='^', s=50)
ax.scatter(
    samples1[:, 0], samples1[:, 1], samples1_probs, c='r', marker='o', s=50)

plt.show()

In [ ]:
net = nn.HybridSequential()
with net.name_scope():
    net.add(
        nn.Conv2D(channels=20, kernel_size=5, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Conv2D(channels=50, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Flatten(),
        nn.Dense(128, activation="relu"),
        nn.Dense(10)
    )

In [ ]:
net.initialize()
net.hybridize()
net

In [ ]:
x = nd.random.normal(shape=(100, 32, 32, 3))

In [ ]:
net(x)

In [ ]:
mx.viz.plot_network(symbol=net, shape={'x':(200, 32, 32, 3)})

In [ ]:
import mxnet as mx

a = mx.sym.Variable('a')
b = mx.sym.Variable('b')
c = mx.sym.add_n(a,b,name="c")
mx.viz.plot_network(symbol=c)